In [1]:
import pandas as pd
import numpy as np


In [2]:
### reading the files
#sys.path.insert(0,'./data_files_ml_engineer/')
user_course_views = pd.read_csv('./data_files_ml_engineer/user_course_views.csv')
user_interests = pd.read_csv('./data_files_ml_engineer/user_interests.csv')
user_assessment_scores = pd.read_csv('./data_files_ml_engineer/user_assessment_scores.csv')
course_tags = pd.read_csv('./data_files_ml_engineer/course_tags.csv')

In [3]:
## for ratings
ratings = user_assessment_scores.copy()
# Drop the date column to draw average scores for user on asessment tags,
# incase if users took assessment multiple times
ratings.drop(['user_assessment_date'],axis = 1, inplace = True)
grouped = ratings.groupby(['user_handle','assessment_tag'])
ratings=grouped.agg('mean')
print(ratings.shape)

(6571, 1)


In [7]:
#ratings.reset_index(inplace=True)
ratings.head()

,user_handle,assessment_tag,user_assessment_score
0,2,photoshop,139
1,4,java,169
2,9,azure-infrastructure-as-a-service,141
3,10,jquery,184
4,11,c-plus-plus,273


- Assuming the course score max is 300 from the data available
    * (0-50)    : 1
    * (51-100)  : 2
    * (101-150) : 3
    * (151-200) : 4
    * (201-250) : 5
    * (251-300) : 6

In [9]:
for indice in ratings.index:
    if 0<=ratings.ix[indice,2]<=50:
        ratings.ix[indice,'rating']=1
        
    if 51<=ratings.ix[indice,2]<=100:
        ratings.ix[indice,'rating']=2
        
    elif 101<=ratings.ix[indice,2]<=151:
        ratings.ix[indice,'rating']=3
        
    if 151<=ratings.ix[indice,2]<=200:
        ratings.ix[indice,'rating']=4
        
    if 201<=ratings.ix[indice,2]<=250:
        ratings.ix[indice,'rating']=5
        
    if 251<=ratings.ix[indice,2]<=300:
        ratings.ix[indice,'rating']=6

In [10]:
del indice

## Things to do
#derive a rating given by a course through course_tag assessment to each user
### 1. check if the course genres and assessment tags are same or if courses and assessment_tags are same
- course_genres and courses in course_tags
- assessment_tags in the user_assessment_scores

In [20]:
course_genres = set(course_tags.course_tags)
print(len(course_genres),' course_genres/course_tags.course_tags available')
courses = set(course_tags.course_id)
print(len(courses),' courses available')
assessment_tags = set(user_assessment_scores.assessment_tag)
print(len(assessment_tags),' : assessement_tags')
print(assessment_tags.issubset(course_genres),' assessment tags is subset of course_tags.course_tags/course_genres')

(999, ' course_genres/course_tags.course_tags available')
(5942, ' courses available')
(54, ' : assessement_tags')
(False, ' assessment tags is subset of course_tags.course_tags/course_genres')


In [13]:
#intersection of available course_genres and users that were assessed on
print(len(assessment_tags.intersection(course_genres)))

16


- creating ids for the course_tags available and assessed

In [14]:
new_tags = assessment_tags.difference(course_genres)

In [15]:
course_tags_updated = course_tags.copy()
course_tags_updated=course_tags_updated.append(pd.DataFrame({'course_id':new_tags,'course_tags':new_tags}))

In [16]:
print(course_tags.shape)
print(course_tags_updated.shape)
empty_df = course_tags_updated[course_tags_updated['course_tags'].isna()]
## updating the empty course_tags with course_ids
print(empty_df.shape)
for indice in empty_df.index:
    course_tags_updated.iloc[indice,1]=course_tags_updated.iloc[indice,0]
    
empty_df = course_tags_updated[course_tags_updated['course_tags'].isna()]
print(empty_df.shape)
del empty_df

(11337, 2)
(11375, 2)
(112, 2)
(0, 2)


In [43]:
course_genres_dict={}
course_genres_dict_reverse={}
course_genres_updated = course_genres.copy()
course_genres_updated.update(new_tags)
print(len(course_genres))
print(len(new_tags))
print(len(course_genres_updated))
for key,value in enumerate(course_genres_updated):
    course_genres_dict[key]=value
    course_genres_dict_reverse[value]=key

999
38
1037


In [44]:
course_genres_df = pd.DataFrame.from_dict(course_genres_dict,orient='index')
course_genres_df.reset_index(inplace=True)
course_genres_df.columns=['course_tag_id','course_tag']
print(course_genres_df.head())

print(course_genres_df.dtypes)

   course_tag_id          course_tag
0              0                 NaN
1              1  visual-studio-code
2              2             msbuild
3              3         data-access
4              4  speech-recognition
course_tag_id     int64
course_tag       object
dtype: object


In [45]:
course_genres_df.dropna(inplace=True)
print(course_genres_df[course_genres_df.course_tag.isna()])

Empty DataFrame
Columns: [course_tag_id, course_tag]
Index: []


In [49]:
ratings['course_tag_id']=ratings.assessment_tag.map(course_genres_dict_reverse)

In [50]:
ratings.head()

,user_handle,assessment_tag,user_assessment_score,rating,course_tag_id
0,2,photoshop,139,3.0,247
1,4,java,169,4.0,652
2,9,azure-infrastructure-as-a-service,141,3.0,1014
3,10,jquery,184,4.0,430
4,11,c-plus-plus,273,6.0,823


In [51]:
ratings_ids = ratings[['user_handle','course_tag_id','rating']]

In [61]:
n_users = user_interests.user_handle.unique().shape[0]
n_users_assessed = int(ratings_ids.user_handle.unique().shape[0])
n_course_tags_assessed = int(ratings_ids.course_tag_id.unique().shape[0])

In [62]:
ratings_ids.dtypes

user_handle        int64
course_tag_id      int64
rating           float64
dtype: object

In [64]:
print(n_course_tags_assessed)
print(n_users_assessed)

54
3114


In [72]:
data_matrix = pd.pivot_table(ratings_ids,index='user_handle',columns='course_tag_id')

In [74]:
data_matrix.fillna(0,inplace=True)

In [75]:
data_matrix=data_matrix.as_matrix()

In [77]:
## finding user similarity
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(data_matrix,metric='cosine')
user_similarity_

In [78]:
course_tag_similarity = pairwise_distances(data_matrix.T,metric='cosine')

#### This gives us the item-item and user-user similarity in an array form. The next step is to make predictions based on these similarities.

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'course':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

user_prediction = predict(data_matrix, user_similarity, type='user')
course_tag_prediction = predict(data_matrix, course_tag_similarity, type='course')

In [91]:
user_similarity

array([[ 0.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  0.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  0., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  0.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  0.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  0.]])

In [92]:
pd.DataFrame(user_similarity).to_csv('./data_files_ml_engineer/trial/user_rec.csv',index=False)

In [93]:
## User similarity based on the levels (beginner, intermediate, advanced)
user_course_views.head()

,user_handle,view_date,course_id,author_handle,level,view_time_seconds
0,1,2017-06-27,cpt-sp2010-web-designers-branding-intro,875,Beginner,3786
1,1,2017-06-28,cpt-sp2010-web-designers-branding-intro,875,Beginner,1098
2,1,2017-06-28,cpt-sp2010-web-designers-css,875,Intermediate,4406
3,1,2017-07-27,cpt-sp2010-web-designers-css,875,Intermediate,553
4,1,2017-09-12,aws-certified-solutions-architect-professional,281,Advanced,102


In [98]:
## Checking for available courses and courses viwed match
courses_viewed = set(rating_views.course_id)

In [99]:
print(courses_viewed.issubset(courses),' : courses viewed are in courses available')

(True, ' : courses viewed are in courses available')


In [100]:
course_available_dict={}
course_available_dict_reverse={}
courses_available = list(courses)
for key,value in enumerate(courses_available):
    course_available_dict[key]=value
    course_available_dict_reverse[value]=key

In [110]:
rating_views.head()

,user_handle,course_ids,level_id
0,1,3529,1
2,1,2905,2
4,1,1247,3
5,1,4857,2
6,1,5134,1


In [107]:
user_course_views['course_ids']=user_course_views.course_id.map(course_available_dict_reverse)
user_course_views['level_id']=user_course_views.level.map({'Beginner':1,'Intermediate':2,'Advanced':3})

In [109]:
rating_views = user_course_views[['user_handle','course_ids','level_id']]

rating_views.drop_duplicates(inplace=True)
print(rating_views[rating_views.course_ids.isna()].shape)

(0, 3)


c:\python27\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [113]:
views_data_matrix = pd.pivot_table(rating_views,columns='course_ids',index='user_handle')
print(views_data_matrix.shape)
print(views_data_matrix.head())

(8760, 5942)
            level_id                                              ...        \
course_ids      0    1    2    3    4    5    6    7    8    9    ...  5932   
user_handle                                                       ...         
1                NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   
2                NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   
3                NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   
4                NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   
5                NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   

                                                          
course_ids  5933 5934 5935 5936 5937 5938 5939 5940 5941  
user_handle                                               
1            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  


In [115]:
views_data_matrix.fillna(0,inplace=True)

In [116]:
views_data_matrix = views_data_matrix.as_matrix()

In [117]:
print(user_course_views.shape)
print(rating_views.shape)

(249238, 8)
(105353, 3)


In [118]:
user_views_similarity = pairwise_distances(views_data_matrix,metric='cosine')

In [120]:
pd.DataFrame(user_views_similarity).to_csv('./data_files_ml_engineer/trial/user_views_similarity.csv',index=False)

In [125]:
courses_available = pd.DataFrame.from_dict(course_available_dict,orient='index')
#courses_available.reset_index(inplace=True)
courses_available.columns=['course']

In [126]:
### user similarity based on the levels can be just done using correlation
courses_available.head()

,course
0,iis-certificates
1,jenkins-introduction
2,preparing-to-migrate-office-365
3,photoshop-cc-layer-styles
4,building-jax-ws-web-services-apache-cxf


In [128]:
rating_views.head()

,user_handle,course_ids,level_id
0,1,3529,1
2,1,2905,2
4,1,1247,3
5,1,4857,2
6,1,5134,1


In [132]:
user_level = pd.DataFrame(rating_views.groupby('user_handle')['level_id'].mean())
user_level['rating_count'] = pd.DataFrame(rating_views.groupby('user_handle')['level_id'].count())
user_level.head()

,level_id,rating_count
user_handle,,
1,1.666667,6
2,1.936842,95
3,1.500000,2
4,1.500000,12
5,1.444444,9


In [133]:
user_level.describe()

,level_id,rating_count
count,8760.000000,8760.000000
mean,1.519516,12.026598
std,0.369866,21.284252
min,1.000000,1.000000
25%,1.250000,2.000000
50%,1.500000,5.000000
75%,1.750000,14.000000
max,3.000000,533.000000


In [137]:
#we have 8760 users, max number of courses user interested in are : 533 and an ntermediate user

In [136]:
user_level.sort_values('rating_count',ascending=False).head()

,level_id,rating_count
user_handle,,
8081,2.105066,533
6800,1.741860,430
8261,1.646154,390
4505,1.634349,361
5162,2.203822,314


In [143]:
8081 in list(rating_views.user_handle)

True

user_views_crosstab.columns.get_level_values(0)

In [163]:
user = int(raw_input('please enter user id :'))
user_views_crosstab = pd.pivot_table(rating_views,columns='user_handle',index='course_ids')

please enter user id :8081


In [165]:
#drop multi index
user_views_crosstab.columns = user_views_crosstab.columns.droplevel()

user_view_ratings = user_views_crosstab[user]
#user_view_ratings[user_view_ratings>0]
similar_users = user_views_crosstab.corrwith(user_view_ratings)

similar_users_df = pd.DataFrame(similar_users,columns=['correlated'])
similar_users_df.dropna(inplace=True)
similar_users_df.head()

,correlated
user_handle,
2,1.0
5,1.0
27,1.0
45,1.0
67,1.0


In [171]:
similar_users_df.tail()

,correlated
user_handle,
9897,1.0
9905,1.0
9907,1.0
9921,1.0
9986,1.0


In [168]:
similar_users_summary = similar_users_df.join(user_level['rating_count'])

In [172]:
#50% users are interested in atleast 5 courses
similar_users_summary[similar_users_summary['rating_count']>5].sort_values('correlated',ascending=False).tail(30)

,correlated,rating_count
user_handle,,
6032,1.0,43
6871,1.0,26
1948,1.0,18
6446,1.0,18
6071,1.0,66
6072,1.0,72
2266,1.0,62
6190,1.0,20
2220,1.0,61
